# Cellule 1: Installation des dépendances

In [ ]:
!pip install -q transformers torch peft bitsandbytes accelerate trl datasets

# Cellule 2: Import des bibliothèques

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# Cellule 3: Connexion à Hugging Face Hub (nécessaire pour Llama, Gemma, etc.)

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# Cellule 4: Configuration du modèle et du dataset

In [ ]:
# Modèle de base que nous allons fine-tuner
model_name = "google/gemma-2-9b-it" # Ou "meta-llama/Llama-3-8B-Instruct"
# Chemin vers votre dataset sur Google Drive
dataset_file = "/content/drive/MyDrive/HAProxy_LLM_Training/haproxy_dataset_qa.jsonl"
# Nouveau nom pour notre modèle fine-tuné
new_model_name = "gemma-2-9b-haproxy-expert"

# Cellule 5: Chargement du dataset

In [ ]:
dataset = load_dataset("json", data_files=dataset_file, split="train")
print(f"Dataset chargé avec {len(dataset)} exemples.")

# Cellule 6: Configuration de la quantification 4-bit (QLoRA)

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# Cellule 7: Chargement du modèle et du tokenizer

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Cellule 8: Formatage du dataset pour le chat template

In [ ]:
def format_chat_template(example):
    message = [
        {"role": "user", "content": example["question"]},
        {"role": "assistant", "content": example["response"]}
    ]
    # L'apply_chat_template formate le message pour le modèle (ex: avec <start_of_turn> etc.)
    text = tokenizer.apply_chat_template(message, tokenize=False)
    return {"text": text}

# On applique le formatage à tout le dataset
dataset = dataset.map(format_chat_template)

# Cellule 9: Configuration de LoRA

In [ ]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"] # Modules pour Gemma/Llama
)

# Cellule 10: Configuration des arguments d'entraînement

In [ ]:
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=50,
    logging_steps=10,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
)

# Cellule 11: Initialisation et lancement de l'entraînement

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)

trainer.train()

# Cellule 12: Sauvegarde de l'adaptateur LoRA

In [ ]:
trainer.model.save_pretrained(new_model_name)
print(f"Adaptateur LoRA sauvegardé sous le nom : {new_model_name}")

# Cellule 13: Test du modèle fine-tuné

In [ ]:
# On charge le modèle de base
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
# On y applique notre adaptateur
model = PeftModel.from_pretrained(base_model, new_model_name)
# On crée un pipeline pour faciliter l'inférence
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

# Question de test
prompt = "Quelle est la directive 'bind' dans HAProxy et comment l'utiliser ?"
result = pipe(f"<start_of_turn>user\n{prompt}<end_of_turn>\n<start_of_turn>model\n")
print(result[0]['generated_text'])